# Import the packages

In [6]:
import pandas as pd
import numpy as np
import torch

# Load the Datasets/Prepare the datasets

In [14]:
# Load the twitter DS
twitter = pd.read_csv('twitter.csv') # https://github.com/t-davidson/hate-speech-and-offensive-language
twitter = twitter[['tweet', 'offensive_language']]
twitter.head(10)

# pip install kaggle

,tweet,offensive_language
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,3
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,3
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,2
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,6
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",2
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",3
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,3
8,""" &amp; you might not get ya bitch back &amp; ...",3
9,""" @rhythmixx_ :hobbies include: fighting Maria...",2


In [15]:





reddit = pd.read_csv('reddit.csv') # https://github.com/jing-qian/A-Benchmark-Dataset-for-Learning-to-Intervene-in-Online-Hate-Speech/tree/master/data
gab = pd.read_csv('gab.csv')

reddit.head()
gab.head()

,id,text,hate_speech_idx,response
0,1. 39869714\n,1. i joined gab to remind myself how retarded ...,[1],"[""Using words that insult one group while defe..."
1,1. 39845588\n2. \t39848775\n3. \t\t39911017\n,1. This is what the left is really scared of. ...,[3],['You can disagree with someones opinion witho...
2,1. 37485560\n2. \t37528625\n,1. It makes you an asshole.\n2. \tGive it to a...,[2],['Your argument is more rational if you leave ...
3,1. 39787626\n2. \t39794481\n,1. So they manage to provide a whole lot of da...,[2],"[""You shouldn't generalize a specific group or..."
4,1. 37957930\n2. \t39953348\n3. \t\t39965219\n,"1. Hi there, i,m Keith, i hope you are doing w...",[3],['If someone is rude it is better to ignore th...


# Build Data Preprocessing Pipeline